In [1]:
%pylab inline
import pandas as pd
pd.set_option('display.precision', 5)

Populating the interactive namespace from numpy and matplotlib


In [2]:
# 静特性および、望目特性の2乗和分解
def staticProp(A,M=1,ve=False,calcSN=False):
    # Degree of Freedom
    Ft = A.size
    Fm = 1
    Fe = Ft - Fm
    # Squared Sum
    St = np.dot(A,A)
    Sm = (A.mean()**2)*A.size # Sm = m*n
    Se = St - Sm
    # Variance
    Vm = Sm/Fm
    Ve = Se/Fe
    # Contribution
    Pm = (Sm-Ve)/St *1.
    Pe = (Se+Ve)/St *1.
    Pt = Pm+Pe
    
    # 品質水準計算のためにVeを返す
    if ve==True:
        return Ve
    
    # calcSNratio[db]
    if calcSN==True:
        eta = ((Sm-Ve)/A.size/M/M)/Ve #eq(8.3)
        return 10*np.log10(eta)
    
    # 有意な偏りmがあるか判定
    if Vm>Ve:
        data = {'F': [Fm,Fe,Ft], 'S':[Sm,Se,St], 'V':[Vm,Ve,nan], 'rho':[Pm,Pe,Pt]}
        df = pd.DataFrame(data=data, index=["m","e","T"])
        print df
    else:
        data = {'F': [Fm,Fe,Ft,Ft], 'S':[Sm,Se,St,St], 'V':[Vm,Ve,nan,St/Ft], 'rho':[Pm,Pe,Pt,Pt]}
        df = pd.DataFrame(data=data, index=["m","e","T","(e)"])
        print df
        print
        print "Vm < Ve より、m を e にプールした"

## 1ヶ月目 - 問2
>ある製品に使われる信号処理用ICの機能限界は$\pm$0.6Vで、それを超えると、ユーザーは修理を依頼するが、その平均損失は3000円である。

---

###1) 製造で規格外になったときの損失は85円/台である。製造の許容差を求めよ。

最適な許容差$x$は、次式を満たすよう"バランス"した場合である。
$$\frac{3000}{0.6^2} = \frac{85}{x^2} $$
これを解くと、

In [3]:
x = 0.6*np.sqrt(85./3000)
print "許容差は、%.2f(V)"%x

許容差は、0.10(V)


###2) 現行製造条件$A_1$、新製造条件$A_2$について、各々12個ずつについて目標値からの差を調べたデータ（単位 V）は次のようであった。$A_1$, $A_2$の品質水準を求めよ。

In [4]:
A1 = np.asarray([0.05, -0.06 , -0.05, 0., 0.06, 0.07, -0.09, 0.06, 0., -0.01, 0.06, 0.05])
A2 = np.asarray([0.09, 0.03, 0.01, 0.07, 0.05, 0.08, 0.06, 0.06, 0.04, 0.04, 0.05, 0.06])

品質水準を次式で計算する
$$ L = \frac{A_0}{\Delta_0^2}\sigma^2 = \frac{A_0}{\Delta_0^2}V_T$$

In [5]:
# 品質水準を計算する
def qualityLevel(A, Ao, Do): 
    Vt= np.dot(A,A)/A.size
    L = Ao/(Do**2)*Vt
    return L

print "A1の品質水準は、%.1f"%qualityLevel(A1,3000,0.6)
print "A2の品質水準は、%.1f"%qualityLevel(A2,3000,0.6)

A1の品質水準は、24.3
A2の品質水準は、27.4


###3) 目標値からの偏りは、工程条件のある因子を調整すれば、平均値を目標値に近づけることができる。調整した時の$A_1$, $A_2$の品質水準を求め、(2)の結果と比較せよ。

In [6]:
print "A1の分散分析表"
staticProp(A1)

A1の分散分析表
      F       S       V   rho
m     1  0.0016  0.0016 -0.04
e    11  0.0334  0.0030  1.04
T    12  0.0350     NaN  1.00
(e)  12  0.0350  0.0029  1.00

Vm < Ve より、m を e にプールした


In [7]:
print "A2の分散分析表"
staticProp(A2)

A2の分散分析表
    F       S       V     rho
m   1  0.0341  0.0341  0.8542
e  11  0.0053  0.0005  0.1458
T  12  0.0394     NaN  1.0000


In [8]:
print "A1は、Vm < Ve より調整できない"
Ve = staticProp(A2,ve=True)
L = 3000/(0.6**2)*Ve
print "A2の調整した時の品質水準は、%.1f"%L

A1は、Vm < Ve より調整できない
A2の調整した時の品質水準は、4.0


###4) 新製造条件の場合、1台あたり17円のコストアップになるとき、平均値を調整したときの$A_1, A_2$の特質を検討せよ。

#### $A_1$の特質
$A_1$に関しては(3)より平均値を調整しない。<br>
したがって$A_1$の品質は(2)で求めた$24.3$(円/台)

#### $A_2$の特質
$A_2$に関しては新製造条件での品質は、<br>
$ 4(Q.Level) + 17(Cost) = 21$(円/台)<br>

したがって、平均値を調整すると$A_2$の特質の方が、<br>
$3$(円/台) 優れている

In [9]:
%load_ext version_information
%version_information numpy, matplotlib, pandas, ipython, version_information

Software versions
Python 2.7.9 64bit [GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.56)]
IPython 3.0.0-dev
OS Darwin 14.4.0 x86_64 i386 64bit
numpy 1.10.0.dev+a02f5c8
matplotlib 1.4.2
pandas 0.16.0
ipython 3.0.0.dev0
version_information 1.0.3
Sun Jul 26 17:22:00 2015 JST